In [135]:
"""Basic demonstration of sweeps and metrics operation."""

# %%
# Imports, etc.

import numpy as np
from functools import partial
import torch


from transformer_lens import HookedTransformer

from activation_additions import (
    prompt_utils,
    utils,
    metrics,
    hook_utils,
    hyperparameter_search
)
from activation_additions.prompt_utils import (
    ActivationAddition,
    pad_tokens_to_match_activation_additions,
    get_block_name,
)
utils.enable_ipython_reload()

# Disable gradients to save memory during inference
_ = torch.set_grad_enabled(False)

from typing import List, Union,Dict
import pandas as pd


In [133]:
# %%
# Load a model
MODEL = HookedTransformer.from_pretrained(model_name="gpt2-xl", device="cpu")
_ = MODEL.to("cuda:0")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda:0


In [107]:
grid_df=layer_coefficient_gridsearch(
    model=MODEL,
    prompts=["The Most beautyful city in the world is"],
    weighted_steering_prompts={" Rome":1," Paris":-1},
    Layer_list=list(range(6,12)),
    coefficient_list=list(range(0,10,2)),
    wanted_completions=" Rome",
    unwanted_completions=" Paris")

In [109]:

perplexity=conditional_perplexity(MODEL,
    MODEL.to_tokens("The only thing we have to fear is"),MODEL.to_tokens(" fear itself")[:, 1:])
perplexity


0.21780490269884467

In [14]:
ActAds =[prompt_utils.ActivationAddition(coeff=1, act_name=1,prompt="hi"),
         prompt_utils.ActivationAddition(coeff=-1, act_name=1,prompt="bye")]
hook_fns=hook_utils.hook_fns_from_activation_additions(MODEL, ActAds)
#for act_name, hook_fn in hook_fns.items():
#    MODEL.add_hook(act_name, hook_fn)

In [137]:
ActAds =[prompt_utils.ActivationAddition(coeff=4, act_name=6,prompt=" hate"),
         prompt_utils.ActivationAddition(coeff=-4, act_name=6,prompt=" love")]
MODEL.remove_all_hook_fns()
perplexity_1=hyperparameter_search.conditional_perplexity(MODEL,
    MODEL.to_tokens("The primary emtion, I have towards you is"),MODEL.to_tokens(" hatred.")[:, 1:])
perplexity_2=hyperparameter_search.conditional_perplexity(MODEL,
    MODEL.to_tokens("The primary emtion, I have towards you is"),MODEL.to_tokens(" hatred.")[:, 1:],ActAds)



TypeError: hook_fn_from_activations.<locals>.prompt_hook() takes from 1 to 2 positional arguments but 3 were given

In [128]:
perplexity_1

5.645045101642609

In [129]:
perplexity_2

5.645045101642609

In [31]:
hook_fns["blocks.1.hook_resid_pre"]

[<function activation_additions.hook_utils.hook_fn_from_activations.<locals>.prompt_hook(resid_pre: jaxtyping.Float[Tensor, 'batch pos d_model'], hook: Optional[transformer_lens.hook_points.HookPoint] = None) -> jaxtyping.Float[Tensor, 'batch pos d_model']>,
 <function activation_additions.hook_utils.hook_fn_from_activations.<locals>.prompt_hook(resid_pre: jaxtyping.Float[Tensor, 'batch pos d_model'], hook: Optional[transformer_lens.hook_points.HookPoint] = None) -> jaxtyping.Float[Tensor, 'batch pos d_model']>]

In [138]:
metric_func=metrics.get_logprob_metric(MODEL, q_funcs=(
                partial(hook_utils.add_hooks_from_dict, hook_fns=hook_fns),
                hook_utils.remove_and_return_hooks
             ))


In [141]:
metric_func([MODEL.to_tokens("The end.")])


logprob_actual_next_token
0  [-2.6872368, -7.175348, -5.065762]

In [79]:
MODEL.remove_all_hook_fns()

In [14]:
ActAds =[prompt_utils.ActivationAddition(coeff=1, act_name=1,prompt="hi"),
         prompt_utils.ActivationAddition(coeff=-1, act_name=1,prompt="bye")]
hook_fns=hook_utils.hook_fns_from_activation_additions(MODEL, ActAds)

In [13]:
MODEL.remove_all_hook_fns()

In [28]:
grid_df=layer_coefficient_gridsearch(
    model=MODEL,
    prompts=["The Most beautyful city in the world is"],
    weighted_steering_prompts={" Rome":1," Paris":-1},
    Layer_list=list(range(6,12)),
    coefficient_list=list(range(0,10,2)),
    wanted_completions=" Rome",
    unwanted_completions=" Paris")

In [22]:
grid_df[0][2]

tensor([[220]], device='cuda:0')

In [23]:
conditional_perplexity(grid_df[0][0],grid_df[0][1],grid_df[0][2])

TypeError: 'list' object is not callable